In [6]:
from src.RunReport import RunReportWeek
        
run = RunReportWeek()  

In [7]:
# copy text from http://results-service.parkrun.com/resultsSystem/App/eventJournoReportHTML.php?evNum=1153   
# can get current event number for here.
text = '''

'''
run.setResultsSystem(text)

# photos
# addPhoto, second param is width, height from the flickr
# third is the section for the photo to appear - options: summary, volunteer, photo
text = '''
[url=https://flic.kr/p/271GkQ3][img]https://farm2.staticflickr.com/1751/42014704334_e4711a669b_z.jpg[/img][/url][url=https://flic.kr/p/271GkQ3]jells_154_18[/url] by [url=https://www.flickr.com/photos/158549992@N02/]Rose Waghorn[/url], on Flickr
'''
run.addPhoto(text, ['640','427'], 'volunteers')
text = '''
[url=https://flic.kr/p/271GkQ3][img]https://farm2.staticflickr.com/1751/42014704334_e4711a669b_z.jpg[/img][/url][url=https://flic.kr/p/271GkQ3]jells_154_18[/url] by [url=https://www.flickr.com/photos/158549992@N02/]Rose Waghorn[/url], on Flickr
'''
run.addPhoto(text, ['427','640'], 'photo')

# Current Event Number, e.g. 154
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=154
# Open url in new browser window, right click and select 'View page source'
# use control-f to search for  id="results"
# copy from the <table to the </h3> on the following line and paste on the blank line between the ''' '''
# <table class="sortable" id="results" 
# to 
# What is this table? </h3>
text = '''

'''
run.setCurrentEventAll(text)

run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 1, e.g. 153
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=153
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 2, e.g. 152
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=152
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 3, e.g. 151
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=151
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 4, e.g. 150
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=150
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 5, e.g. 149
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=149
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 6, e.g. 148
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=148
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

# Current Event Number - 7, e.g. 147
# http://www.parkrun.com.au/jells/results/weeklyresults/?runSeqNumber=147
text = '''

'''
run.addRunners(text)
run.addVolunteers(text)

In [8]:
options = {'pbLimit':2}
html = run.createWeek(3, options)
print(html)
